# ML Assignment 2 | Logistic Regression
### Anirudh Agrawal: 2018A7PS0099H | Aviral Agrawal: 2018A7PS0192H | Vikramjeet Das: 2018A7PS0280H

Lets import some standard libraries and load the dataset first

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

%matplotlib inline
np.random.seed(2022)

In [13]:
data = pd.read_csv('data.csv', header=None)
X = data[[i for i in range(8)]]
y = data[8]

First we'll write some helper functions to shuffle, normalize and split the data into train-test splits

In [14]:
def shuffle(X, y):
    '''
        Shuffles rows of a dataframe and returns shuffled dataframe
    '''
    permute = np.random.permutation(len(X))
    return X[permute], y[permute]

In [15]:
class MinMaxScaler():
    '''
        Class to normalize a pandas dataframe
    '''
    def fit(self, X):
        self.X = X
        self.min = X.min(axis = 0)
        self.max = X.max(axis = 0)
        
    def fit_transform(self, X):
        self.X = X
        self.fit(X)
        return self.transform(X)
    
    def transform(self, X):
        if (isinstance(X, np.ndarray)):
            return ((X - np.array(self.min)) / (np.array(self.max) - np.array(self.min)))
        return (X - self.min) / (self.max - self.min)
        
    def get_params():
        return self.min, self.max

In [16]:
def train_test_split(X, y, test_size=0.1):
    '''
        Splits data into train-test split
            
            Parameters:
                X : Features
                y : Labels
                test_size : Fraction of data to use for test set
            
            Returns:
                (X_train, y_train, X_test, y_test) : Train test split
    '''
    if not isinstance(X, np.ndarray):
        X = np.array(X)
    if not isinstance(y, np.ndarray):
        y = np.array(y)
        
    X, y = shuffle(X, y)
        
    test_indices = np.random.rand(X.shape[0]) < test_size
    return X[~test_indices], y[~test_indices], X[test_indices], y[test_indices]

We normalize the data before processing it further

In [17]:
scaler = MinMaxScaler()
scaled_X = np.array(scaler.fit_transform(X))

In [64]:
class LogisticRegressionGD():
    
    def __init__(self, lamda, include_bias=True):
        self.losses = []
        self.epoch_losses = []
        self.lamda = lamda
        self.include_bias = include_bias
        
    def fit(self, X, y, alpha, max_epochs=500000, regularization=None, show_loss=False):
        if self.include_bias:
            X = self._pad_ones(X)
        self.coeffs = np.zeros((X.shape[1],))
        m = X.shape[0]
        self.losses = []
        epoch = 0
        pbar = tqdm(total=max_epochs)
        while(epoch < max_epochs):
            preds = self._predict(X).reshape(y.shape)
            loss = -(1 / m) * (np.sum((y * np.log(preds)) + (1 - y) * np.log(1 - preds)))
            if regularization == 'l2':
                loss += (self.lamda / m) * (np.sum(np.square(self.coeffs)))
            elif regularization == 'l1':
                loss += ((self.lamda / m) * np.linalg.norm(self.coeffs, ord=1)).reshape((-1, 1))
            self.losses.append(loss)
            
            self.coeffs -= ((alpha / m) * np.sum(np.expand_dims((preds - y), axis=1) * X, axis=0))
            if regularization == 'l2':
                self.coeffs -= (self.lamda * alpha / m) * (2 * self.coeffs)
            elif regularization == 'l1':
                self.coeffs -= ((self.lamda * alpha / m) * (np.sign(self.coeffs))).reshape((-1, 1))
                            
            if (show_loss==True and epoch % 50 == 0):
                print(f'Loss at epoch {epoch}: {self.losses[-1]}')
            if (len(self.losses) > 2 and abs(self.losses[-1] - self.losses[-2]) < 10e-8):
                print('Converged, stopping early.')
                break
            epoch += 1
            pbar.update(1)
        pbar.close()

    def _predict(self, X):
        z = X @ self.coeffs
        return 1 / (1+np.exp(-(z))) - 10e-8 # Subtract eps for numerical stability and avoid log(0) in CE loss
    
    def predict(self, X):
        X = self._pad_ones(X)
        return self._predict(X)
    
    def evaluate(self, X, y):
        preds = self.predict(X)
        preds[preds >= 0.5] = 1
        preds[preds < 0.5] = 0
        return np.sum(preds == y) / len(y)
    
    def _pad_ones(self, X):
        pad_ones = np.ones(X.shape[0]).reshape((-1, 1))
        return np.concatenate((pad_ones, X), axis=1)
    
    def get_coeffs(self):
        return self.coeffs

In [66]:
logregGD = LogisticRegressionGD(lamda=0.001)
logregGD.fit(scaled_X, y, regularization='l2', alpha=0.001, max_epochs=100000, show_loss=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

Loss at epoch 0: 0.6931471201432986
Loss at epoch 50: 0.6913580147835842
Loss at epoch 100: 0.6896561635705141
Loss at epoch 150: 0.6880370684499518
Loss at epoch 200: 0.6864964578582615
Loss at epoch 250: 0.6850302764005858
Loss at epoch 300: 0.6836346748442474
Loss at epoch 350: 0.6823060004395095
Loss at epoch 400: 0.6810407875760365
Loss at epoch 450: 0.6798357487800439
Loss at epoch 500: 0.6786877660542251
Loss at epoch 550: 0.677593882560067
Loss at epoch 600: 0.6765512946400809
Loss at epoch 650: 0.6755573441756968
Loss at epoch 700: 0.6746095112751216
Loss at epoch 750: 0.673705407284255
Loss at epoch 800: 0.6728427681127894
Loss at epoch 850: 0.6720194478668546
Loss at epoch 900: 0.6712334127789873
Loss at epoch 950: 0.6704827354257592
Loss at epoch 1000: 0.6697655892231087
Loss at epoch 1050: 0.6690802431892225
Loss at epoch 1100: 0.6684250569647336
Loss at epoch 1150: 0.667798476079983
Loss at epoch 1200: 0.6671990274591633
Loss at epoch 1250: 0.6666253151512774
Loss at epoc

Loss at epoch 10600: 0.6412593386567145
Loss at epoch 10650: 0.6411787126584853
Loss at epoch 10700: 0.6410981511417138
Loss at epoch 10750: 0.6410176539537082
Loss at epoch 10800: 0.6409372209462791
Loss at epoch 10850: 0.640856851975525
Loss at epoch 10900: 0.6407765469016266
Loss at epoch 10950: 0.6406963055886533
Loss at epoch 11000: 0.6406161279043767
Loss at epoch 11050: 0.6405360137200958
Loss at epoch 11100: 0.6404559629104672
Loss at epoch 11150: 0.6403759753533469
Loss at epoch 11200: 0.6402960509296375
Loss at epoch 11250: 0.6402161895231437
Loss at epoch 11300: 0.6401363910204351
Loss at epoch 11350: 0.6400566553107144
Loss at epoch 11400: 0.6399769822856933
Loss at epoch 11450: 0.6398973718394733
Loss at epoch 11500: 0.6398178238684332
Loss at epoch 11550: 0.6397383382711217
Loss at epoch 11600: 0.6396589149481543
Loss at epoch 11650: 0.6395795538021172
Loss at epoch 11700: 0.6395002547374736
Loss at epoch 11750: 0.6394210176604758
Loss at epoch 11800: 0.6393418424790819
L

Loss at epoch 21050: 0.6256892590716969
Loss at epoch 21100: 0.6256205126408605
Loss at epoch 21150: 0.6255518172642994
Loss at epoch 21200: 0.6254831728923489
Loss at epoch 21250: 0.6254145794753941
Loss at epoch 21300: 0.6253460369638714
Loss at epoch 21350: 0.6252775453082673
Loss at epoch 21400: 0.625209104459119
Loss at epoch 21450: 0.6251407143670139
Loss at epoch 21500: 0.6250723749825898
Loss at epoch 21550: 0.6250040862565356
Loss at epoch 21600: 0.6249358481395892
Loss at epoch 21650: 0.6248676605825401
Loss at epoch 21700: 0.6247995235362269
Loss at epoch 21750: 0.6247314369515395
Loss at epoch 21800: 0.6246634007794173
Loss at epoch 21850: 0.6245954149708497
Loss at epoch 21900: 0.6245274794768767
Loss at epoch 21950: 0.6244595942485879
Loss at epoch 22000: 0.6243917592371231
Loss at epoch 22050: 0.6243239743936724
Loss at epoch 22100: 0.6242562396694752
Loss at epoch 22150: 0.624188555015821
Loss at epoch 22200: 0.6241209203840495
Loss at epoch 22250: 0.62405333572555
Loss

Loss at epoch 31400: 0.6124785601103725
Loss at epoch 31450: 0.6124193938462245
Loss at epoch 31500: 0.6123602693780186
Loss at epoch 31550: 0.6123011866657352
Loss at epoch 31600: 0.6122421456693974
Loss at epoch 31650: 0.6121831463490703
Loss at epoch 31700: 0.612124188664862
Loss at epoch 31750: 0.6120652725769228
Loss at epoch 31800: 0.6120063980454453
Loss at epoch 31850: 0.6119475650306644
Loss at epoch 31900: 0.6118887734928574
Loss at epoch 31950: 0.6118300233923436
Loss at epoch 32000: 0.6117713146894848
Loss at epoch 32050: 0.6117126473446848
Loss at epoch 32100: 0.6116540213183894
Loss at epoch 32150: 0.6115954365710868
Loss at epoch 32200: 0.6115368930633068
Loss at epoch 32250: 0.6114783907556218
Loss at epoch 32300: 0.6114199296086457
Loss at epoch 32350: 0.6113615095830343
Loss at epoch 32400: 0.6113031306394855
Loss at epoch 32450: 0.611244792738739
Loss at epoch 32500: 0.6111864958415764
Loss at epoch 32550: 0.6111282399088206
Loss at epoch 32600: 0.6110700249013368
Lo

...

Loss at epoch 52150: 0.591089680482068
Loss at epoch 52200: 0.5910448776933547
Loss at epoch 52250: 0.5910001032952469
Loss at epoch 52300: 0.5909553572621619
Loss at epoch 52350: 0.5909106395685451
Loss at epoch 52400: 0.5908659501888692
Loss at epoch 52450: 0.590821289097635
Loss at epoch 52500: 0.5907766562693699
Loss at epoch 52550: 0.59073205167863
Loss at epoch 52600: 0.5906874752999977
Loss at epoch 52650: 0.5906429271080837
Loss at epoch 52700: 0.590598407077526
Loss at epoch 52750: 0.5905539151829897
Loss at epoch 52800: 0.5905094513991677
Loss at epoch 52850: 0.5904650157007795
Loss at epoch 52900: 0.5904206080625727
Loss at epoch 52950: 0.5903762284593218
Loss at epoch 53000: 0.5903318768658286
Loss at epoch 53050: 0.5902875532569221
Loss at epoch 53100: 0.5902432576074585
Loss at epoch 53150: 0.5901989898923213
Loss at epoch 53200: 0.5901547500864207
Loss at epoch 53250: 0.5901105381646948
Loss at epoch 53300: 0.5900663541021076
Loss at epoch 53350: 0.5900221978736512
Loss 

Loss at epoch 62550: 0.582346078019886
Loss at epoch 62600: 0.5823066683552766
Loss at epoch 62650: 0.582267282311801
Loss at epoch 62700: 0.5822279198690045
Loss at epoch 62750: 0.5821885810064538
Loss at epoch 62800: 0.5821492657037375
Loss at epoch 62850: 0.5821099739404663
Loss at epoch 62900: 0.5820707056962722
Loss at epoch 62950: 0.5820314609508093
Loss at epoch 63000: 0.5819922396837529
Loss at epoch 63050: 0.5819530418748008
Loss at epoch 63100: 0.5819138675036717
Loss at epoch 63150: 0.5818747165501063
Loss at epoch 63200: 0.5818355889938666
Loss at epoch 63250: 0.5817964848147366
Loss at epoch 63300: 0.5817574039925214
Loss at epoch 63350: 0.5817183465070481
Loss at epoch 63400: 0.5816793123381649
Loss at epoch 63450: 0.5816403014657416
Loss at epoch 63500: 0.5816013138696695
Loss at epoch 63550: 0.5815623495298614
Loss at epoch 63600: 0.5815234084262517
Loss at epoch 63650: 0.5814844905387953
Loss at epoch 63700: 0.5814455958474694
Loss at epoch 63750: 0.581406724332272
Los

Loss at epoch 72850: 0.5746985968094986
Loss at epoch 72900: 0.5746636507771254
Loss at epoch 72950: 0.5746287245788423
Loss at epoch 73000: 0.5745938181981967
Loss at epoch 73050: 0.5745589316187535
Loss at epoch 73100: 0.5745240648240945
Loss at epoch 73150: 0.5744892177978187
Loss at epoch 73200: 0.5744543905235426
Loss at epoch 73250: 0.5744195829848993
Loss at epoch 73300: 0.5743847951655392
Loss at epoch 73350: 0.5743500270491299
Loss at epoch 73400: 0.5743152786193557
Loss at epoch 73450: 0.5742805498599182
Loss at epoch 73500: 0.5742458407545359
Loss at epoch 73550: 0.5742111512869442
Loss at epoch 73600: 0.5741764814408958
Loss at epoch 73650: 0.5741418312001602
Loss at epoch 73700: 0.5741072005485234
Loss at epoch 73750: 0.5740725894697889
Loss at epoch 73800: 0.5740379979477767
Loss at epoch 73850: 0.574003425966324
Loss at epoch 73900: 0.5739688735092846
Loss at epoch 73950: 0.5739343405605293
Loss at epoch 74000: 0.5738998271039452
Loss at epoch 74050: 0.5738653331234368
L

Loss at epoch 83300: 0.5678025810651423
Loss at epoch 83350: 0.5677714470448004
Loss at epoch 83400: 0.5677403297651913
Loss at epoch 83450: 0.5677092292130518
Loss at epoch 83500: 0.5676781453751321
Loss at epoch 83550: 0.5676470782381958
Loss at epoch 83600: 0.56761602778902
Loss at epoch 83650: 0.5675849940143952
Loss at epoch 83700: 0.5675539769011253
Loss at epoch 83750: 0.5675229764360273
Loss at epoch 83800: 0.5674919926059325
Loss at epoch 83850: 0.5674610253976844
Loss at epoch 83900: 0.5674300747981401
Loss at epoch 83950: 0.5673991407941709
Loss at epoch 84000: 0.5673682233726606
Loss at epoch 84050: 0.5673373225205063
Loss at epoch 84100: 0.5673064382246186
Loss at epoch 84150: 0.5672755704719215
Loss at epoch 84200: 0.5672447192493518
Loss at epoch 84250: 0.5672138845438602
Loss at epoch 84300: 0.56718306634241
Loss at epoch 84350: 0.567152264631978
Loss at epoch 84400: 0.567121479399554
Loss at epoch 84450: 0.5670907106321413
Loss at epoch 84500: 0.567059958316756
Loss at

Loss at epoch 93600: 0.5617243467501726
Loss at epoch 93650: 0.5616963995145503
Loss at epoch 93700: 0.5616684665505972
Loss at epoch 93750: 0.5616405478475193
Loss at epoch 93800: 0.5616126433945325
Loss at epoch 93850: 0.5615847531808638
Loss at epoch 93900: 0.5615568771957505
Loss at epoch 93950: 0.5615290154284409
Loss at epoch 94000: 0.5615011678681936
Loss at epoch 94050: 0.5614733345042774
Loss at epoch 94100: 0.5614455153259724
Loss at epoch 94150: 0.5614177103225686
Loss at epoch 94200: 0.5613899194833665
Loss at epoch 94250: 0.5613621427976777
Loss at epoch 94300: 0.5613343802548236
Loss at epoch 94350: 0.5613066318441365
Loss at epoch 94400: 0.5612788975549591
Loss at epoch 94450: 0.5612511773766442
Loss at epoch 94500: 0.5612234712985557
Loss at epoch 94550: 0.5611957793100674
Loss at epoch 94600: 0.5611681014005634
Loss at epoch 94650: 0.5611404375594389
Loss at epoch 94700: 0.5611127877760987
Loss at epoch 94750: 0.5610851520399587
Loss at epoch 94800: 0.5610575303404445


In [ ]:
class PolyRegressionSGD():
    
    def __init__(self, lamda):
        self.losses = []
        self.epoch_losses = []
        self.lamda = lamda
        self.mesh_preds = []
        
    def fit(self, X, y, alpha, max_iters=500000, regularization=None, show_loss=False, surface_gif=False):
        X = self._pad_ones(X)
        self.coeffs = np.zeros((X.shape[1], 1))
        m = X.shape[0]
        self.losses = []
        it = 0
        pbar = tqdm(total=max_iters)
        while(it < max_iters):
            for x, yi in zip(X, y):
                preds = self._predict(X)
                error = preds - y
                loss = (1 / (2 * m)) * np.sum(error ** 2)
                if regularization == 'l2':
                    loss += (self.lamda / m) * (np.sum(np.square(self.coeffs)))
                elif regularization == 'l1':
                    loss += ((self.lamda / m) * np.linalg.norm(self.coeffs, ord=1)).reshape((-1, 1))
                self.losses.append(loss)

                self.coeffs -= (alpha) * ((x.T * (self._predict(x) - yi))).reshape((-1, 1))
                if regularization == 'l2':
                    self.coeffs -= (self.lamda * alpha) * (2 * self.coeffs)
                elif regularization == 'l1':
                    self.coeffs -= ((self.lamda * alpha) * (np.sign(self.coeffs))).reshape((-1, 1))

                it += 1
                pbar.update(1)
                if (show_loss==True and it % 50 == 0):
                    print(f'Loss at iteration {it}: {self.losses[-1]}')
                if (it % 500 == 0 and surface_gif==True):
                    self.mesh_preds.append(self.predict(X_mesh))
            self.epoch_losses.append(self.losses[-1])
            if (len(self.losses) > 2 and abs(self.losses[-1] - self.losses[-2]) < 10e-7):
                print('Converged, stopping early.')
                break
        pbar.close()

    def _predict(self, X):
        return X @ self.coeffs
    
    def predict(self, X):
        X = self._pad_ones(X)
        return self._predict(X)
    
    def evaluate(self, X, y):
        return mse_loss(self.predict(X), y)
    
    def _pad_ones(self, X):
        pad_ones = np.ones(X.shape[0]).reshape((-1, 1))
        return np.concatenate((pad_ones, X), axis=1)
    
    def get_coeffs(self):
        return self.coeffs
    
    def save_loss_gif(self, fname):
            with imageio.get_writer(f'{fname}.gif', mode='I') as writer:
                for i in range(0, 900, 10):
                    print(f'GIFfing: {i}')
                    fig = plt.figure(figsize=(16, 6))
                    ax1 = fig.add_subplot(121, projection='3d')
                    surf = ax1.plot_surface(xx, yy, self.mesh_preds[i].reshape(xx.shape), 
                                            rstride=1, cstride=1, cmap=cm.jet, alpha=0.7)
                    ax1.set_zlim(0, 25000)
                    fig.savefig(f'{fname}.png')
                    image = imageio.imread(f'{fname}.png')
                    writer.append_data(image)
                    plt.close()

In [ ]:
ridgeSGD = PolyRegressionSGD(lamda=0.001)
ridgeSGD.fit(X_full_t, y_full, regularization='l2', alpha=0.001, max_iters=1000000)

In [ ]:
lassoSGD = PolyRegressionSGD(lamda=0.001)
lassoSGD.fit(X_full_t, y_full, regularization='l1', alpha=0.001, max_iters=1000000)

In [ ]:
lassogds = []
lassosgds = []
ridgegds = []
ridgesgds = []
best_lambdas_lasso_gd = []
best_lambdas_lasso_sgd = []
best_lambdas_ridge_gd = []
best_lambdas_ridge_sgd = []
lambdas = np.random.uniform(size=10).tolist()
lambdas.append(0)

X_train, y_train, X_test, y_test = train_test_split(data[feature_cols], y_full, 0.3)
X_val, y_val, X_test, y_test = train_test_split(X_test, y_test, 1/3)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
degrees = [1,2,3,4,5,6,7,8,9,10]
for degree in degrees:
    poly = PolynomialFeatures(degree, include_bias=False)
    X_tr = poly.fit_transform(X_train)
    X_v = poly.transform(X_val)
    X_t = poly.transform(X_test)
    X_mesh = poly.transform(mesh)
    
    best_loss = 100000000000
    best_lambda = 0.01
    for lamda in lambdas:
        ridgeGD = PolyRegressionGD(lamda=lamda)
        ridgeGD.fit(X_tr, y_train, regularization='l2', alpha=0.001, max_epochs=1000000, show_loss=False)
        loss = ridgeGD.evaluate(X_v, y_val)
        if loss < best_loss:
            best_loss = loss
            best_lambda = lamda
            best_model = ridgeGD
        print(f'Ridge : GD : lambda {lamda} : Degree {degree} : Train = {ridgeGD.evaluate(X_tr, y_train)} : Val = {loss} : Test = {ridgeGD.evaluate(X_t, y_test)}')

    ridgegds.append(best_model)
    best_lambdas_ridge_gd.append(best_lambda)
    
    bestloss = 100000000000
    best_lambda = 0.01
    for lamda in lambdas:
        ridgeSGD = PolyRegressionSGD(lamda=lamda)
        ridgeSGD.fit(X_tr, y_train, regularization='l2', alpha=0.001, max_iters=1000000, show_loss=False)
        loss = ridgeSGD.evaluate(X_v, y_val)
        if loss < best_loss:
            best_loss = loss
            best_lambda = lamda
            best_model = ridgeSGD
        print(f'Ridge : SGD : lambda {lamda} : Degree {degree} : Train = {ridgeSGD.evaluate(X_tr, y_train)} : Val = {loss} : Test = {ridgeSGD.evaluate(X_t, y_test)}')

    ridgesgds.append(best_model)
    best_lambdas_ridge_sgd.append(best_lambda)
    
    best_loss = 100000000000
    best_lambda = 0.01
    for lamda in lambdas:
        lassoGD = PolyRegressionGD(lamda=lamda)
        lassoGD.fit(X_tr, y_train, regularization='l1', alpha=0.001, max_epochs=1000000, show_loss=False)
        loss = lassoGD.evaluate(X_v, y_val)
        if loss < best_loss:
            best_loss = loss
            best_lambda = lamda
            best_model = lassoGD
        print(f'Lasso : GD : lambda {lamda} : Degree {degree} : Train = {lassoGD.evaluate(X_tr, y_train)} : Val = {loss} : Test = {lassoGD.evaluate(X_t, y_test)}')

    lassogds.append(best_model)
    best_lambdas_lasso_gd.append(best_lambda)
    
    bestloss = 100000000000
    best_lambda = 0.01
    for lamda in lambdas:
        lassoSGD = PolyRegressionSGD(lamda=lamda)
        lassoSGD.fit(X_tr, y_train, regularization='l1', alpha=0.001, max_iters=1000000, show_loss=False)
        loss = lassoSGD.evaluate(X_v, y_val)
        if loss < best_loss:
            best_loss = loss
            best_lambda = lamda
            best_model = lassoSGD
        print(f'Lasso : SGD : lambda {lamda} : Degree {degree} : Train = {lassoSGD.evaluate(X_tr, y_train)} : Val = {loss} : Test = {lassoSGD.evaluate(X_t, y_test)}')

    lassosgds.append(best_model)
    best_lambdas_lasso_sgd.append(best_lambda)

In [ ]:
for i in range(1, 11):
    poly = PolynomialFeatures(i, include_bias=False)
    X_tr = poly.fit_transform(X_train)
    X_v = poly.transform(X_val)
    X_t = poly.transform(X_test)
    m = ridgegds[i - 1]
    print(f'Best model : Ridge : Degree {i} : GD : lambda {best_lambdas_ridge_gd[i - 1]} : Train loss {m.evaluate(X_tr, y_train)} : Val loss {m.evaluate(X_v, y_val)} : Test loss Train loss {m.evaluate(X_t, y_test)}')
    print()
    m = ridgesgds[i - 1]
    print(f'Best model : Ridge : Degree {i} : SGD : lambda {best_lambdas_ridge_sgd[i - 1]} : Train loss {m.evaluate(X_tr, y_train)} : Val loss {m.evaluate(X_v, y_val)} : Test loss Train loss {m.evaluate(X_t, y_test)}')
    print()
    m = lassogds[i - 1]
    print(f'Best model : Lasso : Degree {i} : GD : lambda {best_lambdas_lasso_gd[i - 1]} : Train loss {m.evaluate(X_tr, y_train)} : Val loss {m.evaluate(X_v, y_val)} : Test loss Train loss {m.evaluate(X_t, y_test)}')
    print()
    m = lassosgds[i - 1]
    print(f'Best model : Lasso : Degree {i} : SGD : lambda {best_lambdas_lasso_sgd[i - 1]} : Train loss {m.evaluate(X_tr, y_train)} : Val loss {m.evaluate(X_v, y_val)} : Test loss Train loss {m.evaluate(X_t, y_test)}')
    print()

In [ ]:
degrees = [1,2,3,4,5,6,7,8,9,10]
for degree in degrees:
    poly = PolynomialFeatures(degree, include_bias=False)
    X_tr = poly.fit_transform(X_train)
    X_v = poly.transform(X_val)
    X_t = poly.transform(X_test)
    X_mesh = poly.transform(mesh)
    
    noregGD = PolyRegressionGD(lamda=0)
    noregGD.fit(X_tr, y_train, alpha=0.001, max_epochs=1000000, show_loss=False)
    loss = noregGD.evaluate(X_v, y_val)
    print(f'Ridge : GD : Degree {degree} : Train = {noregGD.evaluate(X_tr, y_train).round()} : Val = {loss.round()} : Test = {noregGD.evaluate(X_t, y_test).round()}')